**INSTALAMOS LIBRERIAS PARA CARGA**

In [ ]:
! pip install psycopg2
! pip install pgdb 

     |████████████████████████████████| 3.0 MB 17.7 MB/s 
  Created wheel for pgdb: filename=pgdb-0.0.11-py3-none-any.whl size=2800 sha256=f7f8f393f418c8306b8c2e320ce7e6d8ada4f9984671990fda90ac51deebb415
  Stored in directory: /root/.cache/pip/wheels/ff/83/6a/8cd1726f08b2d24a023f99cdb271b156b9d61ab3771ed80e30
Successfully built pgdb


**ASIGNAMOS EL PROYECTO DONDE SE ENCUENTRA NUESTRA BD Y EL BUCKET DE LOS DATOS QUE SE CARGARAN**

In [ ]:
PROJECT_ID = "keepcoding-proyectofinal-01" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
BUCKET_NAME = "kcp-bucket-01" #@param {type:"string"}
REGION = "europe-west1" #@param {type:"string"}

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://kcp-bucket-01/datos_detection/
                                 gs://kcp-bucket-01/datos_equivalencies/
                                 gs://kcp-bucket-01/datos_images_out/
                                 gs://kcp-bucket-01/datos_processed/
                                 gs://kcp-bucket-01/datos_shopping_list/
                                 gs://kcp-bucket-01/datos_store/


**COPIAMOS LA CARPETA DATOS DE NUESTRO BUCKET A NUESTRO LOCAL PARA TRATARLOS ANTES DE REALIZAR LA CARGA**

In [ ]:
! gsutil -m cp -r gs://$BUCKET_NAME/datos_shopping_list /content/

Copying gs://kcp-bucket-01/datos_shopping_list/Datos_Shopping_List_20220328.csv...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                                      


LEEMOS EL FICHERO CSV DE SHOPLIST A UN DATAFRAME

In [ ]:
import os
import pandas as pd
import glob

files_joined = os.path.join('/content/datos_shopping_list', '*.csv')
list_files_store = glob.glob(files_joined)
df_data_shoplist = pd.concat(map(pd.read_csv, list_files_store), ignore_index=True)

COMPROBAMOS LOS DATOS DEL DATAFRAME

In [ ]:
print(df_data_shoplist.columns)
df_data_shoplist.head()

Index(['date_created', 'product', 'qty_pieces_wished'], dtype='object')


,date_created,product,qty_pieces_wished
0,2022-03-22,lemon,10
1,2022-03-22,potato,11
2,2022-03-22,apple,6
3,2022-03-22,banana,7
4,2022-03-22,pear,5


GENERAMOS UN NUEVO CSV CON LOS DATOS CON EL NUEVO FORMATO. APROVECHAMOS PARA QUITAR EL INDICE.

In [ ]:
df_data_shoplist.to_csv('/content/datos_shopping_list/Datos_Shopping_List_format.csv', index=False, sep=',')

**IMPORTAMOS LIBRERIAS PARA TRABAJAR CON LA CARGA DE DATOS**

In [ ]:
import psycopg2
import pgdb

**CARGA DE DATOS EN LA TABLA SHOPLIST**

In [ ]:
def cargar(path):
    """ Conexión al servidor de pases de datos PostgreSQL """
    conexion = None
    try:
        # Lectura de los parámetros de conexion
        #params = config()
 
        # Conexion al servidor de PostgreSQL
        print('Conectando a la base de datos PostgreSQL...')
        conexion = pgdb.Connection(host="34.79.94.25",database="proyecto", user="postgres", password="keepcoding")
 
        # creación del cursor
        cur = conexion.cursor()

        # def_table = '''
        # DROP TABLE IF EXISTS Shoplist;  
        # CREATE TABLE IF NOT EXISTS Shoplist
        # (     date_created DATE NOT NULL,
        #       product TEXT NOT NULL,
        #       qty_pieces_wished NUMERIC NOT NULL
        # );'''
        # cur.execute(def_table)
        # conexion.commit()

        csv_file_name = path
        sql = "COPY Shoplist FROM STDIN DELIMITER ',' CSV HEADER"
        cur.copy_expert(sql, open(csv_file_name, "r"))
        conexion.commit()

        # Cierre de la comunicación con PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conexion is not None:
            conexion.close()
            print('Conexión finalizada.')

In [ ]:
cargar('/content/datos_shopping_list/Datos_Shopping_List_format.csv')

Conectando a la base de datos PostgreSQL...
Conexión finalizada.


**SE MUEVEN LOS DATOS A LA CARPETA PROCESADOS**

In [ ]:
! gsutil mv -r gs://$BUCKET_NAME/datos_shopping_list gs://$BUCKET_NAME/datos_processed

Copying gs://kcp-bucket-01/datos_shopping_list/Datos_Shopping_List_20220328.csv [Content-Type=text/csv]...
Removing gs://kcp-bucket-01/datos_shopping_list/Datos_Shopping_List_20220328.csv...

Operation completed over 1 objects/1.4 KiB.                                      
